# Lab 1 - Basic Image Processing

Many major packages in python provide interfaces to manipulate imagesand videos. The most useful packages are numpy, scipy and scikit-image. All these packages provide the necessary functions and classes to work with signals, images, and develop some machine learning and AI techniques.

The first thing we have to do is load an image to a numpy array that holds all the required information to represent this image. We can represent an 8-bit grayscale image with a size of $n \times m$ pixels, as an array (matrix) $I$ that has size $n \times m$ ($n$ rows and $m$ columns). Each cell of the array $I$ holds a value from $0$ to $255$ which are all the possible values an 8-bit variable can take ($2^8=256$). A value of $0$ means that this pixel is black, and a value of 255 means that the pixel is white.

![Figure1](https://raw.githubusercontent.com/wOOL/COM2028/master/W1/pixelmap_gray.png)

In this figure, we can see a pixelmap of a random grayscale image of size 10x10. The programming structure that holds this image in memory is an array of size 10x10. {Note that the pixel values are all in the range $[0,255]$ with 0 being black and 255 being white.}

![Figure2](https://raw.githubusercontent.com/wOOL/COM2028/master/W1/pixelmap_rgb.png)

In this figure, we can see a pixelmap of a random RGB image of size 10x10. The programming structure that holds this image in memory is an array of size 10x10x3. Each one of the 3 dimensions are similar to the array that holds the data for a grayscale image. The first dimension holds the data regarding the red values of the image, the second of the green values, and the third the values related to the blue colour. There are other colour spaces that represent colours in different ways than the amount of red, green, blue that each pixel has (such as LUV, HSV, HSL etc.) but in this lab we will only use the RGB model. Using other models is straightforward if one knows how to work with RGB. {Each colour is produced by mixing specific weights of the three main components $r,g,b$.} 



## 1. Accessing and manipulating image pixels in python

The first exercise deals with the basics of how to load an image from a file to a numpy array. 

In [ ]:
from skimage.io import imread

In [ ]:
#read a grayscale image to a numpy array
gray = imread('https://raw.githubusercontent.com/wOOL/COM2028/master/W1/lena_gray.png')

In [ ]:
print('gray shape => '+str(gray.shape))

In [ ]:
rgb = imread('https://raw.githubusercontent.com/wOOL/COM2028/master/W1/lena_rgb.png')

In [ ]:
print('rgb shape =>'+str(rgb.shape))

In [ ]:
#print the first pixel values for both images
print('rgb[0,0,:] => '+str(rgb[0,0,:]))
print('gray[0,0] => '+str(gray[0,0]))

The size (shape) of the gray Lena image can also be written as (256,256,1), while the size of the colour RGB Lena image is (256,256,3).

## 2. Converting from RGB to Grayscale

In this exercise we will see how to convert an image from RGB to grayscale, and we will also see how to run a function over all the pixels of an image, avoiding the usual $for$ loops and using some built-in numpy methods.

To convert an image from RGB to grayscale, we have to combine the information that are available in the three channels into a single channel. This is done by taking a combination of the three values r,g,b and producing a new value that represents the intensity. 

So in general the way to compute: 

\begin{equation}
Y = \alpha R + \beta G + \gamma B, \; with \; 0 \leq \alpha,\beta,\gamma \leq 1
\end{equation}

There are many combinations of values that give results that are consistent with the properties of human vision, and two commonly used triads[1] are \{$\alpha = 0.2125$, $\beta =0.7154$, $\gamma =0.0721$\} and \{$\alpha = 0.299$, $\beta =0.587$, $\gamma =0.114$\} {There is also the obvious choice of $\alpha =\beta =\gamma =\frac{1}{3}$ but this is not commonly used.}. 

Of course in order to convert the whole numpy array from an RGB array of size (256,256,3) to a grayscale array of size (256,256) we have to run the conversion formula through all the pixels of the image ($256\times256 = 65536$ conversions).

[1]. ITU-R, Parameter values for the hdtv standards for production and international programme exchange, http://www.itu.int/dms_pubrec/itu-r/rec/bt/R-REC-BT.709-3-199802-S!!PDF-E.pdf 


In [ ]:
import numpy as np
#this is a new import - it is the graphics backend 
#that allow to show and save plots and images.
import matplotlib.pyplot as plt
#need to define the colormap (grayscale - vs rgb)
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
plt.imshow(rgb,cmap=cm.Greys_r,aspect='equal')
plt.show()

In [ ]:
#get the dimensions of the rgb image
(w,h,dims) = rgb.shape
#create an empty array of size (w,h)
grayscale = np.zeros((w,h))

In [ ]:
for x in range(w):
    for y in range(h):
        r = rgb[x,y,0]
        g = rgb[x,y,1]
        b = rgb[x,y,2]
        grayscale[x,y] = 0.2125*r + 0.7154*g + 0.0721*b

In [ ]:
plt.imshow(grayscale,cmap=cm.Greys_r,aspect='equal')
plt.show()

Python itself is not a fast language like C or C++. But the core modules of the numpy and scipy are written in C. That means that everything that involves pure nympy operations, should be more or less as fast as the equivalent compiled C code. Numpy provides many tools that can be used to eliminate the need for pure python code, and these tools should be used when someone writes a computationally expensive program. 

For example, the double $for$ (for each row,column) loop that computes the rgb->grayscale convertion in the above code can be avoided. To do this, we can use the advanced indexing tools of numpy.

We can perform many arithmetical and logical operations directly on the numpy objects. First we have to split the rgb array into its' three components r,g,b and then we can define a new array grayscale which is the same as the result of the double for loop in above code.

In [ ]:
R = rgb[:,:,0] # [:] means all the points in that direction
G = rgb[:,:,1]
B = rgb[:,:,2]
gray = 0.2125*R + 0.7154*G + 0.0721*B

In [ ]:
plt.imshow(gray,cmap=cm.Greys_r,aspect='equal')
plt.show()

With this syntax, the execution time is around 900 times faster than the for loop version. Keep in mind that the R G B are full 2d arrays not scalar values. Is there an increase in speed from loops to indexing?


Indexing is a powerful way to avoid for loops, and it should be used whenever it is possible for fast code. {More about the numpy indexing together with some examples here: http://docs.scipy.org/doc/numpy/user/basics.indexing.html }


## 3. Manipulating images

### 3.1 Image flipping

By manipulating the raw pixels of the image we can perform some effects like image flipping. 

In [ ]:
lena = gray.copy()

In [ ]:
(w,h) = lena.shape
print(w,h)

In [ ]:
#create an empty version of our image
flipped_vertical = np.zeros_like(lena)

In [ ]:
#loop through the whole image and reverse x values
for x in range(w):
    for y in range(h):
        flipped_vertical[w-x-1,y] = lena[x,y]

In [ ]:
#show everything
plt.imshow(flipped_vertical,cmap=cm.Greys_r,aspect='equal')
plt.show()

**Task 1:** Try to eliminate the for loop, by using the numpy advanced indexing. As in numpy A[0,0] means the element in the first row and the first column, and  x[:] means all the elements, x[::-1] means all the elements reversed. 

Use the time measuring snippets to compare the executing speed with the double for loop version (Check https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time for "%%timeit").

In [ ]:
# Complete the code for task 1
flipped_vertical_arr = np.zeros_like(lena)

In [ ]:
#show everything
plt.imshow(flipped_vertical_arr,cmap=cm.Greys_r,aspect='equal')
plt.show()

**Task 2:** Modify the above code so that it flips the image horizontally instead of vertically.


In [ ]:
# Complete the code for task 2
flipped_horizontally_arr = np.zeros_like(lena)

In [ ]:
#show everything
plt.imshow(flipped_horizontally_arr,cmap=cm.Greys_r,aspect='equal')
plt.show()

**Task 3:** If $x$ is the value of any pixel in (0, 255), the inverse can be computed using the function $f(x) = 255 - x$. Modify the code so that it shows the inverse of an image.

In [ ]:
# Complete the code for task 3
inverse = np.zeros_like(lena)

In [ ]:
#show everything
plt.imshow(inverse,cmap=cm.Greys_r,aspect='equal')
plt.show()

### 3.2 Image contrast

We can adjust the contrast of the image, if we multiple each pixel with a value $a$, that we will call $adjustment\; factor$. intuitively, if $a>1$ then we will enhance the image contrast, and if $a<1$ then we will reduce the image contrast. 

One technical problem that we will find this time, is the overflow. When we manipulate 8-bit images, some times our calculations make the pixels get values higher than 255 and lower than 0. For uint8 grayscale images, this will produce an overflow. There are many solutions, but the simplest one is the following:


* Load the image as a float array instead of uint8
* Perform the manipulations
* Convert the values that are above 255 to 255
* Convert the values that are below 0 to 0
* Present the image as a uint8 array

Also in this example, we will learn how to present multiple plots in the same figure using matplotlib's subplot functions.



In [ ]:
#create the new empty versions of our original image
darkened = np.zeros_like(lena)
lightened = np.zeros_like(lena)

In [ ]:
a_darken = 0.2
a_lighten = 1.9

In [ ]:
#set up the a for both cases
for x in range(w):
    for y in range(h):
        lightened[x,y] = a_lighten*lena[x,y]
        darkened[x,y] = a_darken*lena[x,y]

In [ ]:
#fix the overflows
darkened[darkened>255]=255
darkened[darkened<0]=0
lightened[lightened>255]=255
lightened[lightened<0]=0

In [ ]:
fig = plt.figure()
plt.subplot(121, aspect='equal')
plt.imshow(lightened.astype(np.uint8),cmap=cm.Greys_r,aspect='equal',vmin=0,vmax=255)
plt.title('a='+str(a_lighten))
plt.subplot(122, aspect='equal')
plt.imshow(darkened.astype(np.uint8),cmap=cm.Greys_r,aspect='equal',vmin=0,vmax=255)
plt.title('a='+str(a_darken))
plt.show()

**Task 4:** The following codes are the ones that perform the overflow protection. Remove them to see what the result is without overflow handling, and try to interpret it.

```python
darkened[darkened>255]=255
darkened[darkened<0]=0
lightened[lightened>255]=255
lightened[lightened<0]=0
```


In [ ]:
# Complete the code for task 4

### 3.3 Averaging images

To blend images together, we just have to create a new image, where each element is the mean value of the respective elements in the two input images. 

In general, to create a new image $A$, that is a blend of the images $I_1,I_2,I_3,...,I_n$, 

\begin{equation}
A = \sum_{k=1}^{n} w_k \times I_k  \; \textbf{(element-wise)}
\end{equation}

The weights $w_k$ define the amount of contribution of each image in the final averaged image, and the sum of all the weifht should add up to 1.

To demonstrate the averaging of images, we will use the Olivetti dataset which contains 400 face images. We are going to compute how the average face looks like.

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

In [ ]:
# load the dataset with the faces
dataset = fetch_olivetti_faces()

In [ ]:
faces = dataset.data
image_shape = ((64,64))

In [ ]:
n_samples, n_features = faces.shape
new = faces.reshape((400,1,64*64))

In [ ]:
#show the 25 first faces
plt.subplots_adjust(wspace=0.1, hspace=0.1)
for x in range(0, 25):
    image = new[x].reshape((image_shape))
    vmax = max(image.max(), -image.min())
    ax = plt.subplot(5,5,x+1)
    plt.imshow(new[x].reshape((image_shape)),
               cmap=plt.cm.gray,interpolation='nearest',vmin=-vmax, vmax=vmax)
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)
plt.show()

In [ ]:
#main loop for averaging
#the total faces in the db are 400

number_of_faces_to_average = 150

#to get the average wee need to loop over the images,
#sum them up and then divide by the number of elements
average_face = np.zeros(image_shape, dtype=np.float32)

for x in range(number_of_faces_to_average):
    #add to the average
    average_face = np.add(average_face,faces[x].reshape(image_shape))
average_face = np.divide(average_face,number_of_faces_to_average)

In [ ]:
fig = plt.figure()
plt.imshow(average_face, 
           cmap=plt.cm.gray,
           interpolation='nearest',
           vmin=-vmax, 
           vmax=vmax)
plt.show()

**Task 5:** Change the value of the variable **number_of_faces_to_average** to see how it affects the average face image.

In [ ]:
# Complete the code for task 5

### 3.4 Histogram based methods

Histogram is an array that shows the distribution of the pixel values for an image. For an 8-bit grayscale images, there are $2^8=256$ possible different values for each pixel. The histogram of such an image, will consist of an 1D array h of size 256, where h[i]=x means that there are exactly x pixels in the image that take the value i. 


In [ ]:
hist = np.zeros(256)

for (x,y),value in np.ndenumerate(lena.astype(np.uint8)):
    hist[int(value)] += 1

In [ ]:
fig = plt.figure()
plt.subplot(121, aspect='equal')
plt.imshow(lena.astype(np.uint8), cmap=plt.cm.Greys_r,aspect='equal',vmin=0,vmax=255)
plt.title('Image')
plt.xticks([])
plt.yticks([])
plt.subplot(122)
plt.plot(hist)
plt.title('Histogram')
plt.show()

**Task 6:** Plot the 3-component histogram for the colour image Lena. You can create three new grayscale images from the 3 planes of the colour image (r,g,b) and then plot a histogram for each one of them. (Note: You can use fuction 'np.histogram(r.flatten(),256)' to get each component histogram for the colour image Lena)


In [ ]:
#load the rgb version of image
lena = rgb.copy()

In [ ]:
# Complete the code for task 6

**Task 7**: Plot and compare the histogram for the original, flipped and inversed Lena images. 

In [ ]:
# Complete the code for task 7

### 3.5 Histogram equalization

Histogram is a powerful tool because it gives us the general view of the image, and we can use it in order to utilize several transformations on that image. Here we will see how we can use it to make an image with low contrast reveal the information it hides. 

![Figure1](https://raw.githubusercontent.com/wOOL/COM2028/master/W1/moon_original.png)

In this figure, we can see a standard test image for histogram based algorithms, the moon image. The poor contrast makes it difficult to get the details of the image. 

In order to understand the method that is used to improve the quality of this image, we need to be familiar with another thing except the histogram: the cumulative distribution function. The cumulative histogram, supposing the we already have computed the histogram $hist$ can be defined as:

\begin{equation}
c\_hist(x) = \displaystyle\sum\limits_{i=1}^x hist(i)
\end{equation}

This new array F holds information that shows how many pixels in the image have values equal or less to a specific value x. So if we want to find out how many pixels in our image have value less than 200, we need to find the value of $c\_hist(200)$. 

However we can use the information from the cumulative histogram, to make the histogram more spread along the $256$ different values that it can take. First of all, we can to normalise the values of the $c\_hist$ so that they are between $[0,255]$. This can be simply done by 

\begin{equation}
c\_hist_{normalised}(x) = \frac{256 \times c\_hist(x)}{max(c\_hist)}
\end{equation}

If we only keep the integer part for the normalized values, then the values of the $c\_hist_{normalised}(x)$ will be from $[0,255]$. This allows as to use them as a mapping function. So the algorithm that we will use for histogram equalization is just to remap each pixel of the image, to a new value based on the information that $c\_hist_{normalised}(x)$ holds. For example if the $c\_hist_{normalised}(100) = 150$, this means that all the pixels in the original image that have value of 100, should now take the value of 150. 

In [ ]:
from skimage.data import moon
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
#load the moon image: it is a low contrast image
moon = moon()
imhist, _ = np.histogram(moon.flatten(), 256)
#compute the cumulative distribution
cdf = imhist.cumsum()

In [ ]:
#create the pixel equalization transform 
cdf = 255 * cdf / cdf[255] 

In [ ]:
#create a new image to hold the equalized
moon_equalized = np.empty_like(moon)
for (x,y), pixel_value in np.ndenumerate(moon):
    moon_equalized[x,y] = cdf[pixel_value]
    
#get the info for the equalized image
imhist_eq, _ = np.histogram(moon_equalized.flatten(),256)
cdf_eq = imhist_eq.cumsum() 

In [ ]:
#plot the images
plt.figure(0)
plt.subplot(1,2,1)
plt.xticks([])
plt.yticks([])
plt.imshow(moon.astype(np.uint8),
cmap=cm.Greys_r,aspect='equal',vmin=0,vmax=255)
plt.subplot(1,2,2)
plt.xticks([])
plt.yticks([])
plt.imshow(moon_equalized.astype(np.uint8),
cmap=cm.Greys_r,aspect='equal',vmin=0,vmax=255)

#plot the info
plt.figure(1)
plt.subplots_adjust(wspace=1, hspace=1)
x = np.arange(0,256)

plt.subplot(4,1,1)
plt.title('The PDF of the equalized moon image (histogram)')
plt.xlim([0,255])
plt.bar(x,imhist)

plt.subplot(4,1,2)
plt.plot(cdf)
plt.title('The CDF of the equalized moon image')
plt.xlim([0,255])

plt.subplot(4,1,3)
plt.title('The PDF of the equalized moon image (histogram)')
plt.xlim([0,255])
plt.plot(imhist_eq)

plt.subplot(4,1,4)
plt.plot(cdf_eq)
plt.title('The CDF of the equalized moon image')
plt.xlim([0,255])

plt.show()

### 3.6 Convolution based methods

The discrete convolution that we will use in this lab is a process of using with a kernel $K$ of size $n\times n$, on an image $I$ of size $m\times l$ that returns an new image $C$. To compute the value of this new image in the pixel $x,y$ we use the following equation:

\begin{equation}
C(x,y) = \sum\limits_{q=-w}^{w} \sum\limits_{p=-w}^{w} I(x-q,y-p) \cdot K(q,p)
\label{equation:convolution}
\end{equation}

where $w=(n-1)/2$.
The example below clarifies the process of the convolution, using a sample image $I$ and a mask $K$.

\begin{equation} 
I=
\begin{pmatrix}
10 & 20 & 25 & \cdots & \cdots \\
11 & \underline{22} & 33 & \cdots & \cdots \\
44 & 50 & 90 & \cdots & \cdots\\
\cdots & \cdots & \cdots & \cdots & \cdots\\
\cdots & \cdots & \cdots & \cdots & \cdots\\
\end{pmatrix}
\end{equation} 


\begin{equation} 
K= \frac{1}{9} \cdot
\begin{pmatrix}
1 & 2 & 1 \\
0 & 1 & 0 \\
1 & 2 & 1
\end{pmatrix}
\end{equation} 


We want to apply the convolution mask to the pixel which is underlined, with value 22. According to the equation, the new value of this pixel will be:


\begin{equation} 
\frac{1}{9} \{ 1 \cdot 10 + 2 \cdot 20 + 1 \cdot 25 + 0 \cdot 11+ 1 \cdot 22+ 0 \cdot 33+ 1 \cdot 44+ 2 \cdot 50 + 1 \cdot 90 \}= 36
\end{equation} 

So in the new convoluted image that we will call $C$, the element in the same position as the anchor point in the original image (the squared 22), will become 36.

\begin{equation} 
C=
\begin{pmatrix}
\cdots & \cdots & \cdots & \cdots & \cdots \\
\cdots & \underline{36} & \cdots & \cdots & \cdots \\
\cdots & \cdots & \cdots & \cdots & \cdots\\
\cdots & \cdots & \cdots & \cdots & \cdots\\
\cdots & \cdots & \cdots & \cdots & \cdots\\
\end{pmatrix}
\end{equation} 

That means that for an image of size [128,128] and a kernel of size [3,3] to get the convolved image we need to perform $128 \times 128 \times 9$ operations.


In the following code program, you will find a simple implementation of a convolution. The Lena image, is convoluted with the kernel $K$. 


In [ ]:
lena = gray.copy()
(w,h) = lena.shape

In [ ]:
norm_factor = 1.0/21.0 
kernel = norm_factor * np.array([[1, 3, 1],
                   [3, 5, 3], 
                   [1, 3, 1]])  

convoluted = np.empty_like(lena)

In [ ]:
#slow unoptimzed code
for x in range(1,w-1):
    for y in range(2,h-1):
        convoluted[x,y] = kernel[0,0]* lena[x-1,y-1] + kernel[0,1]* lena[x-1,y] 
        + kernel[0,2]* lena[x-1,y+1] + kernel[1,0]* lena[x,y-1] + kernel[1,1]* lena[x,y] 
        + kernel[1,2]* lena[x,y+1] + kernel[2,0]* lena[x+1,y-1] + kernel[2,1]* lena[x+1,y]
        + kernel[2,2]* lena[x+1,y+1]

In [ ]:
#fix the overflows
convoluted[convoluted>255]=255
convoluted[convoluted<0]=0

In [ ]:
#plot the images
plt.figure(0)
plt.subplot(1,2,1)
plt.xticks([])
plt.yticks([])
plt.imshow(lena.astype(np.uint8),
cmap=cm.Greys_r,aspect='equal',vmin=0,vmax=255)
plt.subplot(1,2,2)
plt.xticks([])
plt.yticks([])
plt.imshow(convoluted.astype(np.uint8),
cmap=cm.Greys_r,aspect='equal',vmin=0,vmax=255)

plt.show()

The following code is for the coloured Lena image.

In [ ]:
lena = rgb.copy().astype(np.float32)
(w,h,d) = lena.shape

In [ ]:
norm_factor = 1.0
kernel = norm_factor * np.array([[1, 2, 1],
                   [0, 0, 0], 
                   [-1, -2, -1]]) 
convoluted = np.empty_like(lena)


In [ ]:
#slow unoptimzed code
for x in range(1,w-1):
    for y in range(2,h-1):
        convoluted[x,y] = kernel[0,0]* lena[x-1,y-1] + kernel[0,1]* lena[x-1,y] + kernel[0,2]* lena[x-1,y+1] + kernel[1,0]* lena[x,y-1] + kernel[1,1]* lena[x,y] + kernel[1,2]* lena[x,y+1] + kernel[2,0]* lena[x+1,y-1] + kernel[2,1]* lena[x+1,y]+ kernel[2,2]* lena[x+1,y+1]

In [ ]:
# #fix the overflows
convoluted[convoluted>255]=255
convoluted[convoluted<0]=0

In [ ]:
#plot the images
plt.figure(0)
plt.subplot(1,2,1)
plt.xticks([])
plt.yticks([])
plt.imshow(lena.astype(np.uint8),
           cmap=cm.Greys_r,aspect='equal',vmin=0,vmax=255)
plt.subplot(1,2,2)
plt.xticks([])
plt.yticks([])
plt.imshow(convoluted.astype(np.uint8),
           cmap=cm.Greys_r,aspect='equal')
plt.show()

**Task 8:** Run the code in the colour Lenna image, using the kernels below. You can also experiment with your own kernels, by putting in random numbers and seeing the results.

Below are some important kernels.

\begin{equation} 
motion= \frac{1}{3} \cdot
\begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{pmatrix}
\end{equation} 
\begin{equation} 
sharpen=
\begin{pmatrix}
0 & -1 & 0 \\
-1 & 4 & -1 \\
0 & -1 & 0
\end{pmatrix}
\end{equation} 

\begin{equation} 
laplacian=
\begin{pmatrix}
-1 & -1 & -1 \\
-1 & 8 & -1 \\
-1 & -1 & -1
\end{pmatrix}
\end{equation} 

\begin{equation} 
emboss =
\begin{pmatrix}
-2 & -1 & 0 \\
-1 & 0 & 1 \\
0 & 1 & 2
\end{pmatrix}
\end{equation} 

\begin{equation} 
edge= \frac{1}{8} \cdot
\begin{pmatrix}
1 & 2 & 1 \\
0 & 1 & 0 \\
1 & 2 & 1
\end{pmatrix}
\end{equation} 
\begin{equation} 
gaussian=\frac{1}{21} \cdot
\begin{pmatrix}
1 & 3 & 1 \\
3 & 5 & 3 \\
1 & 3 & 1
\end{pmatrix}
\end{equation} 


In [ ]:
# Complete the code for task 8